In [98]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import numpy as np
import torch
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt

In [100]:
!wget https://raw.githubusercontent.com/mohammedelabbas/SVM/main/Preprocessing.py
!wget https://raw.githubusercontent.com/mohammedelabbas/SVM/main/svm.py
!wget https://raw.githubusercontent.com/mohammedelabbas/SVM/main/word_representation.py

--2022-04-22 08:14:07--  https://raw.githubusercontent.com/mohammedelabbas/SVM/main/Preprocessing.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1270 (1.2K) [text/plain]
Saving to: ‘Preprocessing.py’

Preprocessing.py    100%[===================>]   1.24K  --.-KB/s    in 0s      

2022-04-22 08:14:07 (91.9 MB/s) - ‘Preprocessing.py’ saved [1270/1270]

--2022-04-22 08:14:08--  https://raw.githubusercontent.com/mohammedelabbas/SVM/main/svm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1239 (1.2K) [text/plain]
Saving to: ‘sv

In [58]:
df = pd.read_csv("IMDB Dataset.csv")
df["review"][10]


'Phil the Alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines.<br /><br />At first it was very odd and pretty funny but as the movie progressed I didn\'t find the jokes or oddness funny anymore.<br /><br />Its a low budget film (thats never a problem in itself), there were some pretty interesting characters, but eventually I just lost interest.<br /><br />I imagine this film would appeal to a stoner who is currently partaking.<br /><br />For something similar but better try "Brother from another planet"'

In [59]:
df["sentiment"] [df["sentiment"] == "positive"] = 1
df["sentiment"] [df["sentiment"] == "negative"] = -1
labels  = df["sentiment"]
labels = np.array(labels)

# Data Pre-processing

In [101]:
import re
from Preprocessing import *
preprocessing = DataPreprocessing()
data = preprocessing.preprocess(list(df["review"].iloc[:100]))

NameError: ignored

# Word Representation

In [102]:
from word_representation import WordRepresentation
repr = WordRepresentation()
x= repr.tf_idf(data)
x.shape

3980
100


(100, 3980)

In [82]:
x_train,x_test,y_train,y_test = train_test_split(x,labels[:100], test_size = 0.1, random_state = 10)
x_train = x_train.astype(np.double)
y_train = y_train.astype(np.double)
x_test = x_test.astype(np.double)
y_test = y_test.astype(np.double)
print (x_train.shape)

(90, 3980)


**Count Frequency Encoding**

**TF-IDF**

#Training

In [108]:
!pip install cvxopt
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers
!pwd

/content


In [109]:
from svm import SVM
model = SVM()
model.train(x_train, y_train)



ImportError: ignored

#Testing

In [110]:
y_pred = model.test(x_test)
model.accuracy(y_pred,y_test)


(300, 30263)
(30263, 1) (2700, 1)
249.0


0.83

#Visualization

In [ ]:
plt.figure(figsize=(10, 8))
# Plotting our two-features-space
sns.scatterplot(x=X_train[:, 0], 
                y=X_train[:, 1], 
                hue=y_train, 
                s=8);
# Constructing a hyperplane using a formula.
w = svc_model.coe          # w consists of 2 elements
b = svc_model.intercept_[0]      # b consists of 1 element
x_points = np.linspace(-1, 1)    # generating x-points from -1 to 1
y_points = -(w[0] / w[1]) * x_points - b / w[1]  # getting corresponding y-points
# Plotting a red hyperplane
plt.plot(x_points, y_points, c='r');

# Encircle support vectors
# plt.scatter(svc_model.support_vectors_[:, 0],
#             svc_model.support_vectors_[:, 1], 
#             s=50, 
#             facecolors='none', 
#             edgecolors='k', 
#             alpha=.5);

# Step 2 (unit-vector):
w_hat = Svmoptim_[0] / (np.sqrt(np.sum(SvmOptim_[0] ** 2)))
# Step 3 (margin):
margin = 1 / np.sqrt(np.sum(svc_model.coef_[0] ** 2))
# Step 4 (calculate points of the margin lines):
decision_boundary_points = np.array(list(zip(x_points, y_points)))
points_of_line_above = decision_boundary_points + w_hat * margin
points_of_line_below = decision_boundary_points - w_hat * margin
# Plot margin lines
# Blue margin line above
plt.plot(points_of_line_above[:, 0], 
         points_of_line_above[:, 1], 
         'b--', 
         linewidth=2)
# Green margin line below
plt.plot(points_of_line_below[:, 0], 
         points_of_line_below[:, 1], 
         'g--',
         linewidth=2)